In [ ]:
from set_home_directory import get_project_root_homedir_in_sys_path
project_root, main_dir = get_project_root_homedir_in_sys_path("inter_areal_predictability")
if project_root is None:
    raise RuntimeError(f"Project root not found: ensure a folder named '{project_root}' exists in one of the sys.path entries.")
print("Project root found:", project_root)

import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import collections
import sys
import os 
from matplotlib import patches
import matplotlib.collections as mcoll
from matplotlib.colors import to_rgba
import json

results_dir = os.path.join(project_root,'results/fig_5/')
fig_dir = project_root + '/results/paper_figures/'
save_figs = False

# ensure the results directory exists
os.makedirs(results_dir, exist_ok=True)
os.chdir(project_root)


sys.path.insert(0,os.path.join(main_dir,'utils/'))
sys.path.insert(0,main_dir)

from utils.fig_5_functions import make_mouse_df, fig5_violinplot, plot_corrs_sns,fig5_supp_violinplot, make_monkey_df, add_group_pairwise_stars_2_sets_5_conditions, add_stars_2_sets
from utils.fig_3_functions import color_label
from utils.stats_functions import get_comparison_test_stars
from utils.fig_4_functions import fig4_violinplot, add_violin_custom_colors_3,add_group_pairwise_stars_2_sets



In [ ]:
mouse_stats_path = os.path.join(project_root, 'results/fig_5',f'mouse_stats.pkl')
with open(mouse_stats_path, 'rb') as f:
	mouse_stats = pickle.load(f)


monkey_names = ['L','A','D']
all_monkey_stats = {}
for monkey_name in monkey_names:
    monkey_stats_path = os.path.join(project_root, 'results/fig_5',f'monkey_{monkey_name}_stats.pkl')
    with open(monkey_stats_path, 'rb') as f:
        all_monkey_stats[monkey_name] = pickle.load(f)

## Mouse stimulus vs. gray screen comparisons 

no need to control for frame size as the frames are similar in size

### plotting

In [ ]:
df_mouse_all = make_mouse_df(mouse_stats, dataset_types=['ori32','natimg32','ori32_spont','natimg32_spont'])
df_mouse  = df_mouse_all[df_mouse_all['Mouse Name']!='MP027']#remove mouse 27 since it doesnt have a grey screen period
df_mouse_visual =  df_mouse[(df_mouse['SNR']>2)&(df_mouse['Split-half r']>0.8)] 
df_mouse_nonvisual =  df_mouse[(df_mouse['SNR']<2)&(df_mouse['Split-half r']<0.8)]

In [ ]:
#Fig 5A left
neuron_property='EV'
variable1='Direction'
variable2='Activity Type'
palette = ['#72BEB7','#EDAEAE']

fontsize=7
fig, ax = plt.subplots(figsize=(1.3,1.8))

fig5_violinplot(df_mouse_visual, x=variable1, y=neuron_property, 
                hue=variable2, ax=ax, y_label='EV fraction', linewidth=0,gap=-0.2, width=0.7,cut=0)

color_label(ax, palette,fontsize, x_offset=-0.1, predictor_color='black')
ax.set_title('"visual"\nneurons', fontsize=fontsize*1.1, x=0.42, y=0.8)
if save_figs is True:
    plt.savefig(fig_dir +'fig5_mouse_stimulus_comparisons_EV_visual_neurons.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
#Fig 5A right
fig, ax = plt.subplots(figsize=(1.3,1.8))

fig5_violinplot(df_mouse_nonvisual, x=variable1, y=neuron_property, 
                hue=variable2, ax=ax, y_label='EV fraction', linewidth=0,gap=-0.2, width=0.7,cut=0)
ax.set_title('"non-visual"\nneurons', fontsize=fontsize*1.1, x=0.42, y=0.8)
color_label(ax, palette,fontsize, x_offset=-0.1,y_offset= -0.11, predictor_color='black')
ax.set_ylabel('')
ax.set_yticklabels('')
if save_figs is True:
    plt.savefig(fig_dir +'fig5_mouse_stimulus_comparisons_EV_nonvisual_neurons.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
df_mouse_corr=pd.merge(df_mouse[df_mouse['Activity Type']=='stimulus'],df_mouse[df_mouse['Activity Type']=='gray screen'], on=['Mouse', 'Mouse Name', 'Area',
       'Direction', 'SNR', 'Split-half r','control_shuffle'])
df_mouse_corr_visual =  df_mouse_corr[(df_mouse_corr['SNR']>2)&(df_mouse_corr['Split-half r']>0.8)&(df_mouse_corr['control_shuffle']==False)] 
df_mouse_corr_nonvisual =  df_mouse_corr[(df_mouse_corr['SNR']<2)&(df_mouse_corr['Split-half r']<0.8)&(df_mouse_corr['control_shuffle']==False)]

In [ ]:
#Fig 5B left
figsize=(1.1,1.1)
fig, ax = plt.subplots(figsize=figsize)
area="L2/3"
plot_corrs_sns(df_mouse_corr_visual, x='EV_x', y='EV_y', area=area,
               ax=ax, xy=(0.01,0.75), color='lightseagreen', alpha=0.3, s=5)
ax.set_title(f'{area} "visual"\nneurons', fontsize=8)
plt.show()

In [ ]:
#Fig 5B right
figsize=(1.1,1.1)
fig, ax = plt.subplots(figsize=figsize)
area="L4"
plot_corrs_sns(df_mouse_corr_visual, x='EV_x', y='EV_y', area=area,
               ax=ax, xy=(0.01,0.75), color='lightseagreen', alpha=0.3, s=5)
# ax.set_title(f'{area} "visual"\nneurons', fontsize=8)
ax.set_ylabel('')
ax.set_yticklabels('')
if save_figs is True:
    plt.savefig(fig_dir +'fig5_mouse_corr_ev_images_spont_L4_visual_neurons.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
# Fig 5C left
fig, ax = plt.subplots(figsize=figsize)
area="L2/3"
plot_corrs_sns(df_mouse_corr_nonvisual, x='EV_x', y='EV_y', area=area,
               ax=ax, xy=(0.01,0.75), color='lightseagreen', alpha=0.3, s=5)
# ax.set_title(f'{area} "non-visual"\nneurons', fontsize=8)
if save_figs is True:
    plt.savefig(fig_dir +'fig5_mouse_corr_ev_images_spont_L23_nonvisual_neurons.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
# Fig 5C right
fig, ax = plt.subplots(figsize=figsize)
area="L4"
plot_corrs_sns(df_mouse_corr_nonvisual, x='EV_x', y='EV_y', area=area,
               ax=ax, xy=(0.01,0.75), color='lightseagreen', alpha=0.3, s=5)
# ax.set_title(f'{area} "non-visual"\nneurons', fontsize=8)
ax.set_ylabel('')
ax.set_yticklabels('')
if save_figs is True:
    plt.savefig(fig_dir +'fig5_mouse_corr_ev_images_spont_L4_nonvisual_neurons.pdf',transparent=True,bbox_inches='tight')
plt.show()

### supplemental 

In [ ]:
#Fig Supp 6A left
area='L2/3'
x='Mouse Name'
y='EV'
hue='Activity Type'
y_label='EV fraction'
fontsize=7


fig, ax = plt.subplots(figsize=(2,2))
fig5_supp_violinplot(df_mouse_visual,area, x, y, hue, ax, y_label, linewidth=0,cut=0,
                    fontsize=fontsize,plot_control_ev=True, show_legend=False,leg_loc=(1,0.3))
ax.set_title(f'"visual" {area} neurons', fontsize=fontsize*1.1, x=0.5, y=0.9)
ax.set_ylim(-0.05,0.85)
if save_figs is True:
    plt.savefig(fig_dir +'fig5_supp_all_mice_stimulus_comparisons_EV_visual_neurons_L23.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
#Fig Supp 6B left
area='L4'

fig, ax = plt.subplots(figsize=(2,2))
fig5_supp_violinplot(df_mouse_visual,area, x, y, hue, ax, y_label, linewidth=0,cut=0,
                    fontsize=fontsize,plot_control_ev=True, show_legend=False,leg_loc=(1,0.3))
ax.set_title(f'"visual" {area} neurons', fontsize=fontsize*1.1, x=0.5, y=0.9)
ax.set_ylim(-0.05,0.85)
if save_figs is True:
    plt.savefig(fig_dir +'fig5_supp_all_mice_stimulus_comparisons_EV_visual_neurons_L4.pdf',transparent=True,bbox_inches='tight')

plt.show()

In [ ]:
# Fig Supp 6A right
area='L2/3'

fig, ax = plt.subplots(figsize=(2,2))
fig5_supp_violinplot(df_mouse_nonvisual,area, x, y, hue, ax, y_label, linewidth=0,cut=0,
                    fontsize=fontsize,plot_control_ev=True, show_legend=False,leg_loc=(1,0.3))
ax.set_title(f'"non-visual" {area} neurons', fontsize=fontsize*1.1, x=0.5, y=0.9)
ax.set_ylim(-0.05,0.85)
ax.set_yticklabels('')
ax.set_ylabel(None)
if save_figs is True:
    plt.savefig(fig_dir +'fig5_supp_all_mice_stimulus_comparisons_EV_nonvisual_neurons_L23.pdf',transparent=True,bbox_inches='tight')

plt.show()

In [ ]:
# Fig Supp 6B right
area='L4'

fig, ax = plt.subplots(figsize=(2,2))
fig5_supp_violinplot(df_mouse_nonvisual,area, x, y, hue, ax, y_label, linewidth=0,cut=0,
                    fontsize=fontsize,plot_control_ev=True, show_legend=False,leg_loc=(1,0.3))
ax.set_title(f'"non-visual" {area} neurons', fontsize=fontsize*1.1, x=0.5, y=0.9)
ax.set_ylim(-0.05,0.85)
ax.set_yticklabels('')
ax.set_ylabel(None)
if save_figs is True:
    plt.savefig(fig_dir +'fig5_supp_all_mice_stimulus_comparisons_EV_nonvisual_neurons_L4.pdf',transparent=True,bbox_inches='tight')
plt.show()

## Monkey L stimulus vs. gray screen vs. lights off comparisons

### plotting

In [ ]:
monkey_name = 'L'
monkey_stats = all_monkey_stats[monkey_name]

df_monkey = make_monkey_df(monkey_stats, dataset_types=['SNR','SNR_spont','RS','RS_open','RS_closed'], spont_comparisons=True)
df_monkey_visual =  df_monkey[(df_monkey['SNR']>2)&(df_monkey['Split-half r']>0.8)] 


In [ ]:
neuron_property='EV'
variable1='Direction'
variable2='Activity Type'
palette = ['#72BEB7','#EDAEAE']


In [ ]:
# Fig 5D
fontsize=7
figsize=(4.2,1.8)
fig, ax = plt.subplots(figsize=figsize)
fig5_violinplot(df_monkey_visual, x=variable1, y=neuron_property, 
                hue=variable2, ax=ax, y_label='EV fraction', palette=['#9b9b9b','lightgray','#5b5b5b','#5b5b5b','#5b5b5b'],
                linewidth=0,gap=-0.2, width=0.7,cut=0, animal='monkey', show_legend=False, leg_loc=(1,0.1))
color_label(ax, palette,fontsize, y_offset=-0.14, predictor_color='black')
ax.set_ylim(top=1)
if save_figs is True:
    plt.savefig(fig_dir +'fig5_monkey_stimulus_comparisons_EV.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
df_monkey_corr_ = df_monkey_visual[df_monkey_visual['Activity Type']=='stimulus']
for act_type in df_monkey['Activity Type'].unique()[1:]:
    df_copy = df_monkey_visual[df_monkey_visual['Activity Type']==act_type].copy()
    df_copy.rename(columns={'EV': f'EV {act_type}', 'Activity Type':f'Activity Type {act_type}', 'max corr. val': f'max corr. val{act_type}','Dataset Type':f'Dataset Type {act_type}'}, inplace=True)
    df_monkey_corr_= pd.merge(df_monkey_corr_ ,df_copy, on=['Date', 'Area',
    'Direction', 'SNR', 'Split-half r','control_shuffle','Neuron idx'])
df_monkey_corr = df_monkey_corr_[df_monkey_corr_['control_shuffle']==False]

In [ ]:
#Fig 5E left
figsize=(1.1,1.1)
fig, ax = plt.subplots(figsize=figsize)
area="V4"
plot_corrs_sns(df_monkey_corr, x='EV', y='EV gray screen', area=area,
               ax=ax, xy=(0.01,0.75), alpha=0.3, s=8)
# ax.set_title(f'{area} "visual"\nneurons', fontsize=8)
if save_figs is True:
    plt.savefig(fig_dir +'fig5_monkey_corr_ev_snr_spont_v4.pdf',transparent=True,bbox_inches='tight')
plt.show()



In [ ]:
#Fig 5E right
fig, ax = plt.subplots(figsize=figsize)
area="V1"
plot_corrs_sns(df_monkey_corr, x='EV', y='EV gray screen', area=area,
            ax=ax, xy=(0.01,0.75), alpha=0.3, s=8)
# ax.set_title(f'{area} "visual"\nneurons', fontsize=8)
ax.set_ylabel('')
ax.set_yticklabels('')
if save_figs is True:
    plt.savefig(fig_dir +'fig5_monkey_corr_ev_snr_spont_v1.pdf',transparent=True,bbox_inches='tight')

plt.show()

### supplemental

#### plotting

In [ ]:

data = []
for condition_type in df_monkey_visual['Activity Type'].unique():
    for date in df_monkey_visual.Date.unique():
        for pred_type in df_monkey_visual.Direction.unique():
            data.append({
                'Activity Type':condition_type,
                'Date': date,
                'Direction': pred_type,
                'EV_corr_w_SNR':df_monkey_visual[(df_monkey_visual['Activity Type'] =='stimulus')&(df_monkey_visual.Direction==pred_type)&(df_monkey_visual.Date==date)&(df_monkey_visual.control_shuffle==False)].reset_index()['EV'].corr(df_monkey_visual[(df_monkey_visual['Activity Type'] ==condition_type)&(df_monkey_visual.Direction==pred_type)&(df_monkey_visual.Date==date)&(df_monkey_visual.control_shuffle==False)].reset_index()['EV'])
            })
df_SNR_EV_corr = pd.DataFrame(data) 

In [ ]:
#Fig Supp 6C

directions = ['V1→V4', 'V4→V1']
fig, ax=plt.subplots(figsize=(1.8,1.4))
sns.barplot(df_SNR_EV_corr[df_SNR_EV_corr['Activity Type'].isin(df_SNR_EV_corr['Activity Type'].unique()[1:])], 
            x=variable1, order = directions, y='EV_corr_w_SNR', palette=['lightgray','#5b5b5b','#5b5b5b','#5b5b5b'],
            hue='Activity Type',
            width=0.8, ax=ax,
            err_kws={'linewidth': 1}
            )

custom_colors = ['#B6E3DF','#F1C0C0',
                '#136a66','#a85959',
                '#136a66','#a85959',
                '#136a66','#a85959',]
bars  = [s for s in ax.get_children() if isinstance(s, patches.Rectangle)]
lines = ax.get_lines()
offset_val=0.03
hatch_size=3
for b, (bar, line, color)in enumerate(zip(bars, lines, custom_colors)):
    bar.set_facecolor(color)
    if b==4 or b==5:
        bar.set_hatch('/'*hatch_size)
    if b==6 or b==7:
        bar.set_hatch('.'*hatch_size)
    if b>1:
        line.set_xdata(line.get_xdata()+offset_val)
        bar.set_x(bar.get_x()+offset_val)

hatches = [None,None,'/','.',None]
new_handles = []
for handle, hatch in zip(ax.get_legend_handles_labels()[0], hatches):
    if hatch is not None:
        handle.set_hatch(hatch*hatch_size)
    new_handles.append(handle)
ax.legend(loc=(1.05,0.1), handles = new_handles, fontsize=6)

ax.tick_params(axis='both', labelsize=7,  width=0.5, length=2, pad=2)
ax.set_xticks(ticks=ax.get_xticks(), labels=['V1→V4', 'V4→V1'])
ax.set_ylabel('EV correlation\nw/EV checkerboard', fontsize=7, labelpad=1)
ax.set_xlabel(None)
ax.spines[:].set_linewidth(0.3)
ax.set_ylim(0,1.3)


add_group_pairwise_stars_2_sets_5_conditions(df_SNR_EV_corr[df_SNR_EV_corr["Activity Type"].isin(df_SNR_EV_corr['Activity Type'].unique()[1:])],
                                    neuron_property='EV_corr_w_SNR', x=variable1,
                                    x_order=directions,hue='Activity Type',hue_order=df_SNR_EV_corr['Activity Type'].unique()[1:], 
                                    ax=ax, fontsize=5.5)

sns.despine()
ax.legend_.remove()
if save_figs is True:
    plt.savefig(fig_dir +'fig5_supp_monkey_all_corrs_barplot.pdf',transparent=True,bbox_inches='tight')

plt.show()

In [ ]:
df_monkey_RF = make_monkey_df(monkey_stats, dataset_types=['RF_large','RF_large_spont','RF_thin','RF_thin_spont'], spont_comparisons=True)
df_monkey_RF_visual =  df_monkey[(df_monkey['SNR']>2)&(df_monkey['Split-half r']>0.8)]

In [ ]:
#Fig Supp 6D
figsize=(1.2,1.8)
fontsize=6.7
label_order=['V1→V4', 'V4→V1']
hue_order=['stimulus', 'gray screen']
x='Direction'
y='EV'
hue='Activity Type'
neuron_property = y
fig, ax = plt.subplots(figsize=figsize)

sns.violinplot(x=x, y=y, hue=hue, 
                data=df_monkey_RF_visual[df_monkey_RF_visual['control_shuffle']==False],ax=ax,order=label_order, hue_order=hue_order,
                inner='box',linewidth=0,saturation=1, gap=-.1, width=0.8,
                inner_kws={'box_width':2, 'whis_width':0.5,
                            'marker':'_', 'markersize':3,
                            'markeredgewidth':0.8,
                            },palette=['#9b9b9b','lightgray'],cut=0,
                            )

ax.legend_.remove()
custom_colors = ['#72BEB7','#B6E3DF',
                '#EDAEAE', '#f6d6d6',]
violins  = [s for s in ax.get_children() if isinstance(s, collections.PolyCollection)]
for violin, color in zip(violins, custom_colors):
    violin.set_facecolor(color)

ax.tick_params(axis='both', labelsize=fontsize, width=0.5, length=3, pad=2)
ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=1)
ax.set_xlabel(None)
ax.set_ylim(bottom=-0.05,top=0.98)
sns.despine()
add_stars_2_sets(df_monkey_RF_visual[df_monkey_RF_visual['control_shuffle']==False], neuron_property, variable1, 
                    x_order=label_order,hue=variable2, 
                    ax=ax, fontsize=5.5, mouse_or_date='Date', 
                    height1=0.7, height2=0.7,
                    perm_type='paired')
ax.spines[:].set_linewidth(0.3)
ax.set_title('moving bars', fontsize=fontsize*1.1, x=0.5, y=0.9)
data = df_monkey_RF_visual[df_monkey_RF_visual['control_shuffle']==True][neuron_property]
per_25 = np.percentile(data.dropna().values, 25)
per_75 = np.percentile(data.dropna().values, 75)
ax.axhspan(per_25, per_75, alpha=0.4, color='blue', label='shuffle\ncontrol IQR',
        linewidth=0)
if save_figs is True:
    plt.savefig(fig_dir +'fig5_supp_monkey_stimulus_comparisons_moving_bars_EV.pdf',transparent=True,bbox_inches='tight')
plt.show()



## Monkey A stimulus vs. gray screen vs. lights off comparisons

### plotting

In [ ]:
monkey_name = 'A'
monkey_stats = all_monkey_stats[monkey_name]

df_monkey = make_monkey_df(monkey_stats, dataset_types=['SNR','SNR_spont'], spont_comparisons=True)
df_monkey_visual =  df_monkey[(df_monkey['SNR']>2)&(df_monkey['Split-half r']>0.8)] 


In [ ]:
def get_property_dataset_type_monkey(input_string):
	if 'RF' in input_string:
		return 'Moving Bars'
	
	elif 'SNR' in input_string:
		return 'Checkerboard'
	else:
		return input_string

In [ ]:
#Fig Supp 6E
monkey_name = 'A'
monkey_stats = all_monkey_stats[monkey_name]

df_monkey = make_monkey_df(monkey_stats, dataset_types=['SNR','SNR_spont','RF_thin','RF_thin_spont','RF_large','RF_large_spont'], spont_comparisons=True)
df_monkey['Main Dataset Type']  = df_monkey['Dataset Type'].apply(get_property_dataset_type_monkey)
df_monkey_visual =  df_monkey[(df_monkey['SNR']>2)&(df_monkey['Split-half r']>0.8)] 


monkey_label_dict = {'L':'Monkey L', 'A':'Monkey A', 'D':'Monkey D'}
figsize=(3.4,1.8)
fontsize=7

palette_dict = {'V4':['#72BEB7','#B6E3DF'],'V1':['#EDAEAE', '#f6d6d6']}

x='Main Dataset Type'
hue='Activity Type'
y='EV'


hue_order = df_monkey[hue].unique()
x_order = df_monkey[x].unique()
fig, axes = plt.subplots(figsize=figsize, sharey=True, nrows=1, ncols=2)
for area, ax in zip(['V4','V1'], axes.flatten()):
	mini_df = df_monkey[(df_monkey['Area']==area)&(df_monkey['SNR']>2)&(df_monkey['Split-half r']>0.8)&(df_monkey['control_shuffle']==False)]
	direction = 'V1→V4' if area=='V4' else 'V4→V1'
	sns.violinplot(data=mini_df, y=y, hue=hue, ax=ax, x=x,
				linewidth=0,cut=0,inner_kws={'box_width':2, 'whis_width':0.5,
										'marker':'_', 'markersize':3,
										'markeredgewidth':0.8,
										}, saturation=1,palette=palette_dict[area],
				hue_order=hue_order, order=x_order,
				)
	# compute y for stars a bit above the taller of the pair
	ymins, ymaxs = ax.get_ylim()
	yrange = ymaxs - ymins
	pvals = {}
	for x_i in x_order:
		pvals[x_i] = get_comparison_test_stars(mini_df[mini_df[x]==x_i], independent_variable=hue, neuron_property=y, print_pval=True,
												mouse_or_date='Date',perm_type='paired')
	for i, act in enumerate(x_order):
		if act not in pvals:
			continue
		stars = pvals[act]
		# find max y at this category from plotted data to place stars above
		sub = mini_df[mini_df[x]==act][y].to_numpy()
		sub = sub[np.isfinite(sub)]
		y_top = (np.percentile(sub, 99) if sub.size else ymaxs) + 0.03*yrange
		color = "#C0C0C0" if stars=="n.s." else "black"
		ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=fontsize, color=color)
	
	sns.despine()
	ax.set_xlabel('')
	# ax.set_xticklabels(labels=[monkey_label_dict[l] for l in x_order], rotation=0, fontsize=fontsize)
	ax.tick_params(labelsize=fontsize, pad=1)
	ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=1)
	ax.set_title(f'{direction}', fontsize=fontsize*1.2)
	if ax.legend_:
		ax.legend(title='', fontsize=fontsize*0.8, loc=(0.47,0.83))
		ax.legend_.get_frame().set_linewidth(0.3)
	# frame width
	ax.spines[:].set_linewidth(0.5)
	ax.set_ylim(-0.01,0.85)
	data = df_monkey[(df_monkey['Area']==area)&(df_monkey['SNR']>2)&(df_monkey['Split-half r']>0.8)&(df_monkey['control_shuffle']==True)][y]
	per_25 = np.percentile(data.dropna().values, 25)
	per_75 = np.percentile(data.dropna().values, 75)
	ax.axhspan(per_25, per_75, alpha=0.4, color='blue', label='shuffle\ncontrol IQR',
			linewidth=0,
			)
plt.subplots_adjust(wspace=0.05, hspace=0.3, left=0.1, right=0.95, top=0.9, bottom=0.2)
if save_figs is True:
	plt.savefig(fig_dir + f'fig5_monkey_{monkey_name}_stimulus_comparisons_EV.pdf',transparent=True,bbox_inches='tight')

In [ ]:
df_monkey_corr_ = df_monkey_visual[df_monkey_visual['Activity Type']=='stimulus']
for act_type in df_monkey['Activity Type'].unique()[1:]:
    df_copy = df_monkey_visual[df_monkey_visual['Activity Type']==act_type].copy()
    df_copy.rename(columns={'EV': f'EV {act_type}', 'Activity Type':f'Activity Type {act_type}', 'max corr. val': f'max corr. val{act_type}','Dataset Type':f'Dataset Type {act_type}'}, inplace=True)
    df_monkey_corr_= pd.merge(df_monkey_corr_ ,df_copy, on=['Date', 'Area',
    'Direction', 'SNR', 'Split-half r','control_shuffle'])
df_monkey_corr = df_monkey_corr_[df_monkey_corr_['control_shuffle']==False]

In [ ]:
# Fig Supp 6I left
figsize=(1.1,1.1)
fig, ax = plt.subplots(figsize=figsize)
area="V4"
plot_corrs_sns(df_monkey_corr, x='EV', y='EV gray screen', area=area,
               ax=ax, xy=(0.01,0.75), alpha=0.3, s=8)
# ax.set_title(f'{area} "visual"\nneurons', fontsize=8)
if save_figs is True:
    plt.savefig(fig_dir +f'fig5_monkey_{monkey_name}_corr_ev_snr_spont_v4.pdf',transparent=True,bbox_inches='tight')
plt.show()



In [ ]:
# Fig Supp 6I right
fig, ax = plt.subplots(figsize=figsize)
area="V1"
plot_corrs_sns(df_monkey_corr, x='EV', y='EV gray screen', area=area,
            ax=ax, xy=(0.01,0.75), alpha=0.3, s=8)
# ax.set_title(f'{area} "visual"\nneurons', fontsize=8)
ax.set_ylabel('')
ax.set_yticklabels('')
if save_figs is True:
    plt.savefig(fig_dir +f'fig5_monkey_{monkey_name}_corr_ev_snr_spont_v1.pdf',transparent=True,bbox_inches='tight')

plt.show()

## Monkey D stimulus vs. gray screen vs. lights off comparisons

### plotting

In [ ]:
monkey_name = 'D'
monkey_stats = all_monkey_stats[monkey_name]
threshold = 0.6
df_monkey = make_monkey_df(monkey_stats, dataset_types=['SNR','SNR_spont','RS_open',], spont_comparisons=True)
df_monkey_visual =  df_monkey[(df_monkey['SNR']>2)&(df_monkey['Split-half r']>threshold)] 


In [ ]:
# Fig Supp 6F
neuron_property='EV'
fig, ax = plt.subplots(figsize=((1.5,1.8)))
fontsize=7
label_order=['V1→V4', 'V4→V1']
variable1='Direction'
variable2='Activity Type'
hue_order =df_monkey_visual[variable2].unique()
fig4_violinplot(df_monkey_visual[df_monkey_visual['control_shuffle']==False], x=variable1, y=neuron_property, 
                hue=variable2, ax=ax, label_order=label_order, hue_order=hue_order,
                gap=-0.1,width=0.7,cut=0,
                linewidth=0, saturation=1, palette=['lightseagreen','lightcoral','#72BEB7']
                )
ax.legend_.remove()
sns.despine()
ax.set_ylabel('EV Fraction', fontsize=fontsize, labelpad=0)
ax.tick_params(axis='both', labelsize=fontsize, width=0.3, length=2, pad=1)
sns.despine()
ax.spines[:].set_linewidth(0.3)
add_violin_custom_colors_3(ax, custom_colors=['#72BEB7','#B6E3DF','#136a66',
                '#EDAEAE', '#f6d6d6','#a85959',])
add_group_pairwise_stars_2_sets(df_monkey_visual[df_monkey_visual['control_shuffle']==False], neuron_property, x=variable1, x_order=label_order,
                       hue=variable2, hue_order=hue_order, ax=ax, fontsize=fontsize,height1=0.85, height2=0.95, height3=0.85,
                       stars1_positions=[0.16,0.23,0.30],
                       stars2_positions=[0.69,0.77,0.83])
data = df_monkey_visual[df_monkey_visual['control_shuffle']==True]['EV']
per_25 = np.percentile(data.dropna().values, 25)
per_75 = np.percentile(data.dropna().values, 75)
ax.axhspan(per_25, per_75, alpha=0.1, color='blue', label='shuffle\ncontrol IQR',
        linewidth=0,
           )
# Get the y-axis ticks
y_ticks = plt.gca().get_yticks()
# Check if 1 is among the ticks
if 1 in y_ticks:
    ax.set_ylim(top=1)




# the two violins you want to hatch
target_hex = ['#136a66', '#a85959']      # dark green, dark burgundy
target_rgba = [to_rgba(h) for h in target_hex]

def same_color(a, b, tol=1e-3):
    return np.allclose(a[:3], b[:3], atol=tol) and abs(a[3]-b[3]) < tol

for pc in ax.findobj(mcoll.PolyCollection):
    # some artists return an array of facecolors, take the first
    fc = pc.get_facecolor()
    if len(fc): 
        fc = tuple(fc[0])
        if any(same_color(fc, t) for t in target_rgba):
            pc.set_hatch('///')
            # pc.set_edgecolor('k')     # helps hatch show up
            # pc.set_linewidth(0.4)
# color_label(ax, palette,fontsize, y_offset=-0.05, predictor_color='black')

ax.set_ylim(top=0.39)
if save_figs is True:
    plt.savefig(fig_dir +f'fig5_monkey_{monkey_name}_stimulus_comparisons_EV.pdf',transparent=True,bbox_inches='tight')

plt.show()


In [ ]:
df_monkey_corr_ = df_monkey_visual[df_monkey_visual['Activity Type']=='stimulus']
for act_type in df_monkey['Activity Type'].unique()[1:]:
    df_copy = df_monkey_visual[df_monkey_visual['Activity Type']==act_type].copy()
    df_copy.rename(columns={'EV': f'EV {act_type}', 'Activity Type':f'Activity Type {act_type}', 'max corr. val': f'max corr. val{act_type}','Dataset Type':f'Dataset Type {act_type}'}, inplace=True)
    df_monkey_corr_= pd.merge(df_monkey_corr_ ,df_copy, on=['Date', 'Area',
    'Direction', 'SNR', 'Split-half r','control_shuffle'])
df_monkey_corr = df_monkey_corr_[df_monkey_corr_['control_shuffle']==False]

In [ ]:
# Fig Supp 6J left
figsize=(1.1,1.1)
fig, ax = plt.subplots(figsize=figsize)
area="V4"
plot_corrs_sns(df_monkey_corr, x='EV', y='EV gray screen', area=area,
               ax=ax, xy=(0.0,0.2), alpha=0.3, s=12, xlim=(-0.01, 0.21), ylim=(-0.01, 0.21),
               tick_values = [0,0.1])
# ax.set_title(f'{area} "visual"\nneurons', fontsize=8)
if save_figs is True:
    plt.savefig(fig_dir + f'fig5_monkey_{monkey_name}_corr_ev_snr_spont_v4.pdf',transparent=True,bbox_inches='tight')
plt.show()



In [ ]:
# Fig Supp 6J right
fig, ax = plt.subplots(figsize=figsize)
area="V1"
plot_corrs_sns(df_monkey_corr, x='EV', y='EV gray screen', area=area,
            ax=ax, xy=(0.0,0.2), alpha=0.3, s=12, xlim=(-0.01, 0.21), ylim=(-0.01, 0.21),
               tick_values = [0,0.1])
# ax.set_title(f'{area} "visual"\nneurons', fontsize=8)
ax.set_ylabel('')
ax.set_yticklabels('')
if save_figs is True:
    plt.savefig(fig_dir +f'fig5_monkey_{monkey_name}_corr_ev_snr_spont_v1.pdf',transparent=True,bbox_inches='tight')

plt.show()

## Monkey $L_{A}$ stimulus vs. gray screen  comparisons

### plotting

In [ ]:

with open(os.path.join(project_root, 'results/fig_4', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)


monkey_label_dict = {'L':'Monkey L', 'A':'Monkey A', 'D':'Monkey D'}
palette_dict = {'V4':['#72BEB7','#B6E3DF'],'V1':['#EDAEAE', '#f6d6d6']}


def get_property_dataset_type_monkey(input_string):
	if 'RF' in input_string:
		return 'Moving Bars'
	
	elif 'SNR' in input_string:
		return 'Checkerboard'
	else:
		return input_string

x='Main Dataset Type'
hue='Activity Type'
y='EV'

figsize=(3.4,1.8)
fontsize=7

main_monkey_name = 'L'
subsample_monkey_name = 'A'


df_monkey_L_A = pd.DataFrame()
p_vals_L_A = {}

for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	all_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas']]
	df_monkey = make_monkey_df(subsampled_monkey_stats, dataset_types=all_datataset_types_, spont_comparisons=True)
	df_monkey['Main Dataset Type']  = df_monkey['Dataset Type'].apply(get_property_dataset_type_monkey)
	df_monkey_L_A = pd.concat([df_monkey_L_A, df_monkey], ignore_index=True)
	
	hue_order = df_monkey[hue].unique()
	x_order = df_monkey[x].unique()

	for area in ['V4','V1']:
		if area not in p_vals_L_A:
			p_vals_L_A[area] = {}
		mini_df = df_monkey[(df_monkey['Area']==area)&(df_monkey['SNR']>2)&(df_monkey['Split-half r']>0.8)&(df_monkey['control_shuffle']==False)]
		direction = 'V1→V4' if area=='V4' else 'V4→V1'
		
		pvals = {}
		for x_i in x_order:
			pvalue, pvals[x_i] = get_comparison_test_stars(mini_df[mini_df[x]==x_i], independent_variable=hue, neuron_property=y, print_pval=True,
													mouse_or_date='Date',perm_type='paired', return_pval=True)
			if x_i not in p_vals_L_A[area]:
				p_vals_L_A[area][x_i] = []
			p_vals_L_A[area][x_i].append(pvalue)

In [ ]:
# take median over seeds for p_vals_L_A
p_vals_L_A_median = {}
for area in p_vals_L_A:
	p_vals_L_A_median[area] = {}
	for x_i in p_vals_L_A[area]:
		p_vals_L_A_median[area][x_i] = np.median(p_vals_L_A[area][x_i])

In [ ]:
# Fig Supp 6H
def p_to_stars(p):
	return "***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "n.s."))
hue_order = df_monkey_L_A[hue].unique()
x_order = df_monkey_L_A[x].unique()
fig, axes = plt.subplots(figsize=figsize, sharey=True, nrows=1, ncols=2)
for area, ax in zip(['V4','V1'], axes.flatten()):
	mini_df = df_monkey_L_A[(df_monkey_L_A['Area']==area)&(df_monkey_L_A['SNR']>2)&(df_monkey_L_A['Split-half r']>0.8)&(df_monkey_L_A['control_shuffle']==False)]
	direction = 'V1→V4' if area=='V4' else 'V4→V1'
	sns.violinplot(data=mini_df, y=y, hue=hue, ax=ax, x=x,
				linewidth=0,cut=0,inner_kws={'box_width':2, 'whis_width':0.5,
										'marker':'_', 'markersize':3,
										'markeredgewidth':0.8,
										}, saturation=1,palette=palette_dict[area],
				hue_order=hue_order, order=x_order,
				)
	# compute y for stars a bit above the taller of the pair
	ymins, ymaxs = ax.get_ylim()
	yrange = ymaxs - ymins

	for i, act in enumerate(x_order):
		if act not in p_vals_L_A_median[area]:
			continue
		pval = p_vals_L_A_median[area][act]
		stars = p_to_stars(pval)
		# find max y at this category from plotted data to place stars above
		sub = mini_df[mini_df[x]==act][y].to_numpy()
		sub = sub[np.isfinite(sub)]
		y_top = (np.percentile(sub, 99) if sub.size else ymaxs) + 0.03*yrange
		color = "#C0C0C0" if stars=="n.s." else "black"
		ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=fontsize, color=color)
	
	sns.despine()
	ax.set_xlabel('')
	# ax.set_xticklabels(labels=[monkey_label_dict[l] for l in x_order], rotation=0, fontsize=fontsize)
	ax.tick_params(labelsize=fontsize, pad=1)
	ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=1)
	ax.set_title(f'{direction}', fontsize=fontsize*1.2)
	if ax.legend_:
		ax.legend(title='', fontsize=fontsize*0.8, loc=(0.47,0.83))
		ax.legend_.get_frame().set_linewidth(0.3)
	# frame width
	ax.spines[:].set_linewidth(0.5)
	ax.set_ylim(-0.01,0.85)
	data = df_monkey[(df_monkey['Area']==area)&(df_monkey['SNR']>2)&(df_monkey['Split-half r']>0.8)&(df_monkey['control_shuffle']==True)][y]
	per_25 = np.percentile(data.dropna().values, 25)
	per_75 = np.percentile(data.dropna().values, 75)
	ax.axhspan(per_25, per_75, alpha=0.4, color='blue', label='shuffle\ncontrol IQR',
			linewidth=0,
			)
	ax.legend_.remove()
plt.subplots_adjust(wspace=0.05, hspace=0.3, left=0.1, right=0.95, top=0.9, bottom=0.2)
if save_figs is True:
	plt.savefig(fig_dir + f'fig5_monkey_L_subsampled_A_stimulus_comparisons_EV.pdf',transparent=True,bbox_inches='tight')

## Monkey $L_{D}$ subsampled

In [ ]:

with open(os.path.join(project_root, 'results/fig_5', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)


monkey_label_dict = {'L':'Monkey L', 'A':'Monkey A', 'D':'Monkey D'}
palette_dict = {'V4':['#72BEB7','#B6E3DF'],'V1':['#EDAEAE', '#f6d6d6']}

fontsize=7
label_order=['V1→V4', 'V4→V1']
variable1='Direction'
variable2='Activity Type'
neuron_property='EV'

main_monkey_name = 'L'
subsample_monkey_name = 'D'

threshold = 0.6
df_monkey_L_D = pd.DataFrame()

pval_dict_all_seeds = {}
for s, seed in enumerate(subsample_seeds):
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	all_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas']]
	
	df_monkey = make_monkey_df(subsampled_monkey_stats, dataset_types=['SNR','SNR_spont','RS_open',], spont_comparisons=True)
	df_monkey_visual =  df_monkey[(df_monkey['SNR']>2)&(df_monkey['Split-half r']>threshold)].reset_index()
	df_monkey_L_D = pd.concat([df_monkey_L_D, df_monkey_visual], ignore_index=True)
	hue_order =df_monkey_visual[variable2].unique()
	
	pvals_dict = add_group_pairwise_stars_2_sets(df_monkey_visual[df_monkey_visual['control_shuffle']==False], neuron_property, x=variable1, x_order=label_order,
						hue=variable2, hue_order=hue_order, ax=ax, fontsize=fontsize,height1=0.85, height2=0.95, height3=0.85,
						stars1_positions=[0.16,0.23,0.30],
						stars2_positions=[0.69,0.77,0.83], return_p_values=True, sort_keys=['Date','Neuron idx'], perm_type='paired')
	pval_dict_all_seeds[seed] = pvals_dict




In [ ]:
def p_to_stars(p):
	return "***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "n.s."))
pval_dict_median_across_seeds = {}
for direction in label_order:
	if direction not in pval_dict_median_across_seeds:
		pval_dict_median_across_seeds[direction] = {}
		pval_dict_median_across_seeds[direction]['pvals'] = []
		pval_dict_median_across_seeds[direction]['labels'] = pval_dict_all_seeds[subsample_seeds[0]][f'{label_order[0]}_labels']
	pvals_all_seeds = []
	for seed, pval_dicts in pval_dict_all_seeds.items():
		pvals_all_seeds.append(pval_dicts[f'{direction}_pvals'])
	pval_dict_median_across_seeds[direction]['pvals'] = np.median(np.array(pvals_all_seeds), axis=0)
	pval_dict_median_across_seeds[direction]['stars'] = [p_to_stars(p) for p in pval_dict_median_across_seeds[direction]['pvals']]


In [ ]:
# Fig Supp 6G
fig, ax = plt.subplots(figsize=((1.5,1.8)))
fig4_violinplot(df_monkey_L_D[df_monkey_L_D['control_shuffle']==False], x=variable1, y=neuron_property, 
					hue=variable2, ax=ax, label_order=label_order, hue_order=hue_order,
					gap=-0.1,width=0.7,cut=0,
					linewidth=0, saturation=1, palette=['lightseagreen','lightcoral','#72BEB7']
					)
ax.legend_.remove()
sns.despine()
ax.set_ylabel('EV Fraction', fontsize=fontsize, labelpad=0)
ax.tick_params(axis='both', labelsize=fontsize, width=0.3, length=2, pad=1)
sns.despine()
ax.spines[:].set_linewidth(0.3)
add_violin_custom_colors_3(ax, custom_colors=['#72BEB7','#B6E3DF','#136a66',
				'#EDAEAE', '#f6d6d6','#a85959',])
add_group_pairwise_stars_2_sets(df_monkey_L_D[df_monkey_L_D['control_shuffle']==False], neuron_property, x=variable1, x_order=label_order,
					hue=variable2, hue_order=hue_order, ax=ax, fontsize=fontsize,height1=0.85, height2=0.95, height3=0.85,
					stars1_positions=[0.16,0.23,0.30],
					stars2_positions=[0.69,0.77,0.83], return_p_values=False, stars_dict=pval_dict_median_across_seeds)
pval_dict_all_seeds[seed] = pvals_dict
data = df_monkey_L_D[df_monkey_L_D['control_shuffle']==True]['EV']
per_25 = np.percentile(data.dropna().values, 25)
per_75 = np.percentile(data.dropna().values, 75)
ax.axhspan(per_25, per_75, alpha=0.1, color='blue', label='shuffle\ncontrol IQR',
		linewidth=0,
		)
# Get the y-axis ticks
y_ticks = plt.gca().get_yticks()
# Check if 1 is among the ticks
if 1 in y_ticks:
	ax.set_ylim(top=1)

# the two violins you want to hatch
target_hex = ['#136a66', '#a85959']      # dark green, dark burgundy
target_rgba = [to_rgba(h) for h in target_hex]

def same_color(a, b, tol=1e-3):
	return np.allclose(a[:3], b[:3], atol=tol) and abs(a[3]-b[3]) < tol

for pc in ax.findobj(mcoll.PolyCollection):
	# some artists return an array of facecolors, take the first
	fc = pc.get_facecolor()
	if len(fc): 
		fc = tuple(fc[0])
		if any(same_color(fc, t) for t in target_rgba):
			pc.set_hatch('///')
			# pc.set_edgecolor('k')     # helps hatch show up
			# pc.set_linewidth(0.4)
# color_label(ax, palette,fontsize, y_offset=-0.05, predictor_color='black')
ax.set_ylim(top=0.73)

if save_figs is True:
	plt.savefig(fig_dir + f'fig5_monkey_L_subsampled_{subsample_monkey_name}_stimulus_comparisons_EV.pdf',transparent=True,bbox_inches='tight')